In [2]:
import os
import json
import rasterio
import urllib.request
import pystac

from datetime import datetime, timezone
from shapely.geometry import Polygon, mapping
from tempfile import TemporaryDirectory

In [3]:
tmp_dir = TemporaryDirectory()
img_path = os.path.join(tmp_dir.name, 'masam2_minconc40_source_20140127_v2.png')

In [4]:
url = ('https://noaadata.apps.nsidc.org/'
       'NOAA/G10005_V2/BrowseSource/2014/'
       'masam2_minconc40_source_20140127_v2.png')
urllib.request.urlretrieve(url, img_path)

('/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/masam2_minconc40_source_20140127_v2.png',
 <http.client.HTTPMessage at 0x10f605690>)

In [5]:
catalog = pystac.Catalog(id='practice-catalog', description='This catalog is a basic demonstration catalog utilizing an from Maisie.')

In [6]:
print(list(catalog.get_children()))
print(list(catalog.get_items()))

[]
[]


In [7]:
print(json.dumps(catalog.to_dict(), indent=4))

{
    "type": "Catalog",
    "id": "practice-catalog",
    "stac_version": "1.0.0",
    "description": "This catalog is a basic demonstration catalog utilizing an from Maisie.",
    "links": []
}


In [8]:
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [9]:
# Run the function and print out the results
bbox, footprint = get_bbox_and_footprint(img_path)
print("bbox: ", bbox, "\n")
print("footprint: ", footprint)

bbox:  [0.0, 2550.0, 2100.0, 0.0] 

footprint:  {'type': 'Polygon', 'coordinates': (((0.0, 2550.0), (0.0, 0.0), (2100.0, 0.0), (2100.0, 2550.0), (0.0, 2550.0)),)}


/Users/roma8902/mambaforge/envs/noaa-ai/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [10]:
datetime_utc = datetime.now(tz=timezone.utc)

In [11]:
# populate pystac item
item = pystac.Item(id='local-image',
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime_utc,
                 properties={})

In [12]:
print(item.get_parent() is None)

True


In [13]:
catalog.add_item(item)

<Link rel=item target=<Item id=local-image>>

In [14]:
item.get_parent()

<Catalog id=practice-catalog>

In [15]:
catalog.describe()

* <Catalog id=practice-catalog>
  * <Item id=local-image>


In [16]:
# Add Asset and all its information to Item 
item.add_asset(
    key='image',
    asset=pystac.Asset(
        href=img_path,
        media_type=pystac.MediaType.PNG
    )
)

In [17]:
print(json.dumps(item.to_dict(), indent=4))

{
    "type": "Feature",
    "stac_version": "1.0.0",
    "id": "local-image",
    "properties": {
        "datetime": "2024-04-22T15:34:48.801937Z"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    0.0,
                    2550.0
                ],
                [
                    0.0,
                    0.0
                ],
                [
                    2100.0,
                    0.0
                ],
                [
                    2100.0,
                    2550.0
                ],
                [
                    0.0,
                    2550.0
                ]
            ]
        ]
    },
    "links": [
        {
            "rel": "root",
            "href": null,
            "type": "application/json"
        },
        {
            "rel": "parent",
            "href": null,
            "type": "application/json"
        }
    ],
    "assets": {
        "image": {
 

In [18]:
print(catalog.get_self_href() is None)
print(item.get_self_href() is None)

True
True


In [19]:
catalog.normalize_hrefs(os.path.join(tmp_dir.name, "stac"))

In [20]:
print("Catalog HREF: ", catalog.get_self_href())
print("Item HREF: ", item.get_self_href())

Catalog HREF:  /var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/catalog.json
Item HREF:  /var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/local-image/local-image.json


In [21]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [22]:
!ls {tmp_dir.name}/stac/*

/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/catalog.json

/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/local-image:
local-image.json


In [23]:
with open(catalog.self_href) as f:
    print(f.read())

{
  "type": "Catalog",
  "id": "practice-catalog",
  "stac_version": "1.0.0",
  "description": "This catalog is a basic demonstration catalog utilizing an from Maisie.",
  "links": [
    {
      "rel": "root",
      "href": "./catalog.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./local-image/local-image.json",
      "type": "application/json"
    }
  ]
}


In [24]:
with open(item.self_href) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2024-04-22T15:34:48.801937Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          0.0,
          2550.0
        ],
        [
          0.0,
          0.0
        ],
        [
          2100.0,
          0.0
        ],
        [
          2100.0,
          2550.0
        ],
        [
          0.0,
          2550.0
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/masam2_minconc40_source_20140127_v2.png",
      "type": "image/png"
    }
  },
  "bbox": [
    0.0,
    2550.0,
    2100.0,
    0.0
  ],
  "stac_extensions": []
}


In [25]:
catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)


In [26]:
with open(item.get_self_href()) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2024-04-22T15:34:48.801937Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          0.0,
          2550.0
        ],
        [
          0.0,
          0.0
        ],
        [
          2100.0,
          0.0
        ],
        [
          2100.0,
          2550.0
        ],
        [
          0.0,
          2550.0
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/catalog.json",
      "type": "application/json"
    },
    {
      "rel": "self",
      "href": "/var/folders/dk/dk0501kx2wn6g0b3cjgyhmk80000gp/T/tmpa7aqi60w/stac/local-image/local-image.json",
      "type": "application/

In [27]:
catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [28]:
with open(item.get_self_href()) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2024-04-22T15:34:48.801937Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          0.0,
          2550.0
        ],
        [
          0.0,
          0.0
        ],
        [
          2100.0,
          0.0
        ],
        [
          2100.0,
          2550.0
        ],
        [
          0.0,
          2550.0
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "../../masam2_minconc40_source_20140127_v2.png",
      "type": "image/png"
    }
  },
  "bbox": [
    0.0,
    2550.0,
    2100.0,
    0.0
  ],
  "stac_extensions": []
}


In [29]:
tmp_dir.cleanup()